In [2]:
!pip install telethon nltk scikit-learn



  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 702.2/702.2 kB 7.1 MB/s eta 0:00:00
  Created wheel for pyaes: filename=pyaes-1.6.1-py3-none-any.whl size=26346 sha256=06e92221389268dd8430a629a73c344e17e062e1f3e440f1b2eba56665517407
  Stored in directory: /root/.cache/pip/wheels/d6/84/5f/ea6aef85a93c7e1922486369874f4740a5642d261e09c59140
Successfully built pyaes


In [11]:
import nest_asyncio
nest_asyncio.apply()

import os
import asyncio
from telethon import TelegramClient
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
import re
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer

# Download NLTK resources
nltk.download('vader_lexicon')

class TelegramStockSentimentAnalyzer:
    def __init__(self, api_id, api_hash, phone_number):
        """
        Initialize Telegram client and sentiment analyzer
        """
        self.client = TelegramClient(phone_number, api_id, api_hash)
        self.sia = SentimentIntensityAnalyzer()

    async def fetch_messages(self, channel_username, limit=100):
        """
        Fetch messages from a specific Telegram channel
        """
        await self.client.start()
        messages = []

        try:
            async for message in self.client.iter_messages(channel_username, limit=limit):
                if message and message.text:
                    messages.append(message.text)
        except Exception as e:
            print(f"Error fetching messages: {e}")
        finally:
            await self.client.disconnect()

        return messages

    def preprocess_text(self, text):
        """
        Clean and preprocess text data
        """
        text = re.sub(r'[^a-zA-Z\s]', '', str(text).lower())  # Remove special characters
        return text.strip()

    def extract_sentiment(self, text):
        """
        Perform sentiment analysis on text
        """
        return self.sia.polarity_scores(str(text))['compound']

    def prepare_dataset(self, messages):
        """
        Prepare dataset for machine learning
        """
        preprocessed_messages = [self.preprocess_text(msg) for msg in messages]
        sentiments = [self.extract_sentiment(msg) for msg in messages]
        labels = [1 if sentiment > 0 else 0 for sentiment in sentiments]
        return preprocessed_messages, labels

    def train_model(self, messages):
        """
        Train machine learning model
        """
        X, y = self.prepare_dataset(messages)

        # Check if we have enough data
        if len(X) < 10:
            print("Insufficient data for training. Need more messages.")
            return None

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        vectorizer = TfidfVectorizer(max_features=5000)
        X_train_vec = vectorizer.fit_transform(X_train)
        X_test_vec = vectorizer.transform(X_test)
        clf = RandomForestClassifier(n_estimators=100, random_state=42)
        clf.fit(X_train_vec, y_train)
        y_pred = clf.predict(X_test_vec)
        return {
            'accuracy': accuracy_score(y_test, y_pred),
            'classification_report': classification_report(y_test, y_pred)
        }

# Async function to fetch and train
async def fetch_and_train():
    # Replace with your actual Telegram API credentials
    API_ID = 27936508  # Your API ID
    API_HASH = '6e486e35236141d71e1ef8eedd72da26'  # Your API Hash
    PHONE_NUMBER = '+918287321467'  # Your phone number
    CHANNEL_USERNAME = 'https://t.me/USDT_InvestmentInsightsga5oI'  # Telegram channel username

    # Create analyzer
    analyzer = TelegramStockSentimentAnalyzer(API_ID, API_HASH, PHONE_NUMBER)

    # Fetch messages
    messages = await analyzer.fetch_messages(CHANNEL_USERNAME, limit=100)

    # Train model
    if messages:
        model_results = analyzer.train_model(messages)

        if model_results:
            print("Model Performance:")
            print(f"Accuracy: {model_results['accuracy']}")
            print("Classification Report:")
            print(model_results['classification_report'])
        else:
            print("Model training failed. Insufficient data.")
    else:
        print("No messages fetched from the channel.")

# Wrapper to run async function in notebook
def run_telegram_analysis():
    loop = asyncio.get_event_loop()
    return loop.run_until_complete(fetch_and_train())

# Run the analysis
run_telegram_analysis()

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


Model Performance:
Accuracy: 0.6
Classification Report:
              precision    recall  f1-score   support

           0       0.33      0.14      0.20         7
           1       0.65      0.85      0.73        13

    accuracy                           0.60        20
   macro avg       0.49      0.49      0.47        20
weighted avg       0.54      0.60      0.55        20



In [12]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive
